* profiling spectrum-flux landscape
* explicit symmetry constraining
* https://doi.org/10.1038/s41534-021-00382-6

In [1]:
from DiSuQ.Torch import models
from DiSuQ.Torch.optimization import OrderingOptimization,PolynomialOptimization
from DiSuQ.Torch.optimization import lossTransition
from torch import tensor, float32 as float
from numpy import arange,linspace,array
from DiSuQ.utils import plotCompare,plotHeatmap,plotOptimization
import pickle
from torch import set_num_threads
set_num_threads(32)

In [2]:
with open('target_fluxqubit.p', 'rb') as content: target_info = pickle.load(content)
target_spectrum = target_info['spectrum']

In [3]:
E10 = target_spectrum[:,1] - target_spectrum[:,0]
E20 = target_spectrum[:,2] - target_spectrum[:,0]

In [4]:
target = {'E10':E10[[0,20,-1]],'E20':E20[[0,20,-1]]}

In [5]:
basis = [1,1,1]; rep = 'Q'
basis = [10,10]
basis = {'O':[4],'J':[8,8],'I':[]}; rep = 'K'
circuit = models.shuntedQubit(basis,sparse=False)
print(circuit.circuitComponents())

{'JJ1': 120.0, 'C1': 10.0, 'JJ2': 50.000003814697266, 'C2': 50.000003814697266, 'JJ3': 120.0, 'C3': 10.0, 'I': 100.0}


In [6]:
circuit.modeDistribution()

(1, 0, 2)

In [7]:
flux_range = tensor(linspace(0,1,3))
flux_profile = [[flux] for flux in flux_range]
flux_point = ('I')

#### pre-optimization profile

In [8]:
H_LC = circuit.kermanHamiltonianLC()

In [9]:
H_J = circuit.kermanHamiltonianJosephson

In [10]:
H = H_LC+H_J({'I':tensor(.5)})
H.shape

torch.Size([1156, 1156])

In [11]:
pre_E0,pre_Ex = circuit.spectrumManifold(flux_point,flux_profile,H_LC,H_J,excitation=[1,2])

In [12]:
pre_Ex = pre_Ex.detach().numpy()

In [ ]:
plotCompare(flux_range,{'Ist Excitation':pre_Ex[0],'IInd Excitation':pre_Ex[1],'Anharmonicity':pre_Ex[1]-2*pre_Ex[0]},
            'Shunted-Excitation','$\Delta \Phi_{ext}$','E(GHz)')

## optimization

In [ ]:
flux_profile = [{'I':flux} for flux in flux_range]

In [ ]:
lossObjective = lossTransition(tensor(target['E10'],dtype=float),tensor(target['E20'],dtype=float))
optim = OrderingOptimization(circuit,representation='K')

In [16]:
dLogs,dParams,dCircuit = optim.optimization(lossObjective,flux_profile,iterations=50,lr=.0001)

In [17]:
plotCompare(dLogs.index,dLogs,'Optimizing Flux Profile - Shunted Qubit','iteration')

In [18]:
plotCompare(dCircuit.index,dCircuit,'Circuit Parameters','iteration')

#### post-optimization profile

In [19]:
H_LC = circuit.kermanHamiltonianLC()

In [20]:
H_J = circuit.kermanHamiltonianJosephson

In [21]:
H = H_LC+H_J({'I':tensor(.5)})
H.shape

torch.Size([1156, 1156])

In [22]:
flux_profile = [[flux] for flux in flux_range]
flux_point = ('I')

In [23]:
post_E0,post_Ex = circuit.spectrumManifold(flux_point,flux_profile,H_LC,H_J,excitation=[1,2])

In [24]:
post_Ex = post_Ex.detach().numpy()

In [43]:
plotCompare(flux_range,{'post Ist':post_Ex[0],'post IInd':post_Ex[1],
                        'pre Ist':pre_Ex[0],'pre IInd':pre_Ex[1],
                        'Target01':target['E10'],'Target20':target['E20'],'Anharmonicity':post_Ex[1]-2*post_Ex[0]},
            'Shunted-Excitation','external flux','E(GHz)')

## Loss Scape

In [26]:
print(circuit.circuitComponents())
static = {'C1': 10.0,
          'JJ2': 50.0, 'C2': 50.000003814697266,
           'C3': 10.0,
          'I': 100.0}

{'JJ1': 125.91297149658203, 'C1': 4.230661392211914, 'JJ2': 38.333335876464844, 'C2': 96.83409118652344, 'JJ3': 118.36480712890625, 'C3': 11.195722579956055, 'I': 66.77928924560547}


In [27]:
JJ1 = linspace(123,128,100,endpoint=True)
JJ3 = linspace(116,121,100,endpoint=True)

In [28]:
scape = {'JJ1':JJ1,'JJ3':JJ3}

In [29]:
flux_profile = [{'I':flux} for flux in flux_range]

In [30]:
Loss = optim.lossScape(lossObjective,flux_profile,scape,static)

In [31]:
path = {'SGD':dCircuit[['JJ1','JJ3']].to_numpy()}

In [41]:
import importlib
import DiSuQ
importlib.reload(DiSuQ.utils)

<module 'DiSuQ.utils' from '/Users/chishti/JupyterHub/DiSuQ/utils.py'>

In [42]:
plotOptimization(Loss,JJ1,JJ3,path,'Loss - shunted Qubit profiling','JJ1','JJ3',export='pdf')